In [1]:
import numpy as np

import multiprocessing as mp

In [2]:
# def alpha_estimator(m, X):
#     N = len(X)
#     n = int(N/m) # must be an integer
#     Y = np.sum(X.reshape(n, m),1)
#     eps = np.spacing(1)

#     Y_log_norm =  np.log(np.abs(Y) + eps).mean()
#     X_log_norm =  np.log(np.abs(X) + eps).mean()
#     diff = (Y_log_norm - X_log_norm) / np.log(m)
#     return 1 / diff

# def hill_estimator(X,k):
#     X = np.log(np.abs(X)) #assuming symmetric
#     Xs = np.sort(X)
#     term1 = np.mean(Xs[-(k+1):])
#     term2 = Xs[k]
#     return 1/(term1-term2)

In [3]:
# Corollary 2.4 in Mohammadi 2014 - for multi-d
def alpha_estimator(m, X):
    # X is N by d matrix
    N = len(X)
    n = int(N/m) # must be an integer
    Y = np.sum(X.reshape((n, m, -1)), 1)
    eps = np.spacing(1)
    Y_log_norm = np.log(np.linalg.norm(Y, axis=1) + eps).mean()
    X_log_norm = np.log(np.linalg.norm(X, axis=1) + eps).mean()
    diff = (Y_log_norm - X_log_norm) / np.log(m)
    return 1/diff

In [4]:
# number of data points
n = 10000
# dimension of x
d = 100
# batch size
batch = 10

sig_w = 3
sig_x = 1
sig_y = 3

# std_noise = 3
w = np.random.normal(0,3,d)
# w = sig_w * np.random.randn(d,1)
X = sig_x * np.random.randn(n,d)
Y = X@w.reshape(-1,1) + sig_y * np.random.randn(n,1)

In [5]:
w

array([[0.49913832]])

In [24]:
(np.dot(Xk,w0)).shape

(10,)

In [20]:
np.dot(np.transpose(Xk), (Yk-np.dot(Xk,w0)))

array([[1553.28688832, 1599.83429978, 1552.4751576 , 1653.06332266,
        1608.00416658, 1413.40637158, 1441.95281649, 1348.91756913,
        1668.96388044, 1545.73517396]])

In [5]:
def loss_batch(w,X,Y,batch):
    idx = np.random.randint(Y.shape[0], size = batch)
    X1 = X[idx,:]
    Y1 = Y[idx,:]
    loss = np.sum(np.square(X1@w -Y1))/(2*batch)
    return loss

def loss(w,X,Y):
    loss = np.sum(np.square(np.dot(X,w) -Y))/(2*X.shape[0])
    return loss

def linearreg(d,X,Y,K,stepsize,batch,w_star):
    loss_list = []
    w_list = []
    
    w0 = np.random.uniform(-3,3,d)
    w_list.append(w0)
    loss_list.append(loss_batch(w0,X,Y,batch))
    
    for k in range(K):
        
        idx = np.random.randint(Y.shape[0], size = batch)
        Xk = X[idx,:]
        Yk = Y[idx,:]
        w = w_list[-1].reshape(-1,1) - stepsize / batch * (Xk.T @ (Xk @ w_list[-1].reshape(-1,1) - Yk))
        w_list.append(w.reshape(-1))
        
        loss_list.append(loss_batch(w,X,Y,batch))
        
    w_list = np.array(w_list)
    loss_list = np.array(loss_list)
    
    w_mean = np.mean(w_list[-500:], axis = 0)
    
    return w_mean

def linearreg_unif(d,X,Y,K,base_lr,max_lr,batch,w_star):
    loss_list = []
    w_list = []
    
    w0 =np.random.uniform(-3,3,d)
    w_list.append(w0)
    loss_list.append(loss_batch(w0,X,Y,batch))
    
    for k in range(K):
        
        stepsize = np.random.uniform(low=base_lr, high=max_lr)
        idx = np.random.randint(Y.shape[0], size = batch)
        Xk = X[idx,:]
        Yk = Y[idx,:]
#         temp = Xk @ w_list[-1].reshape(-1,1) - Yk
#         temp = Xk.T @ temp
#         print(temp.shape)
        
        w = w_list[-1].reshape(-1,1) - stepsize / batch * (Xk.T @ (Xk @ w_list[-1].reshape(-1,1) - Yk))
#         print((Xk @ w_list[-1].reshape(-1,1) )shape)
        w_list.append(w.reshape(-1))
        
        loss_list.append(loss_batch(w,X,Y,batch))
        
    w_list = np.array(w_list)
    loss_list = np.array(loss_list)
    
    w_mean = np.mean(w_list[-500:], axis = 0)
    w_final = w_list[-1] - w_star
    
    return w_mean, w_final, loss_list

def linearreg_cyc(d,X,Y,K,base_lr,max_lr,stepsize,batch,w_star):
    loss_list = []
    w_list = []
    
    w0 =np.random.uniform(-3,3,d)
    w_list.append(w0)
    loss_list.append(loss_batch(w0,X,Y,batch))
    
    for k in range(K):
        
        cycle = np.floor(1 + (k+1) / (2*stepsize))
        loc = np.abs((k+1) / stepsize - 2*cycle +1)
        lr = base_lr + (max_lr - base_lr) * max(0,(1-loc))
#         print(lr)
        
        idx = np.random.randint(Y.shape[0], size = batch)
        Xk = X[idx,:]
        Yk = Y[idx,:]
        w = w_list[-1].reshape(-1,1) - lr / batch * (Xk.T @ (Xk @ w_list[-1].reshape(-1,1) - Yk))
        w_list.append(w.reshape(-1))
        
        loss_list.append(loss_batch(w,X,Y,batch))
        
    w_list = np.array(w_list)
    loss_list = np.array(loss_list)
    
    w_mean = np.mean(w_list[-500:], axis = 0)
    w_final = w_list[-1] - w_star
    
    return w_mean, w_final

def linearreg_mc(d,X,Y,K,base_lr,max_lr,p,stepsize,batch,w_star):
    loss_list = []
    w_list = []
    
    onestep = (max_lr-base_lr)/stepsize
    
    w0 = np.random.uniform(-3,3,d)
    w_list.append(w0)
    loss_list.append(loss_batch(w0,X,Y,batch))
#     lr_list=[]
    
    for k in range(K):
        
        if k == 0:
            lr = base_lr
        elif k == 1:
            lr = base_lr + onestep
        else:
            if lr <= base_lr:
                p = max(p, 1-p)
                lr = base_lr + onestep
            elif lr >= max_lr:
                p = min(p,1-p)
                lr = max_lr - onestep
            else:
                temp = np.random.uniform(0,1)
                if temp <= p:
                    lr = min(max_lr, lr+onestep)
                else:
                    lr = max(base_lr, lr-onestep)
#         lr_list.append(lr)
        idx = np.random.randint(Y.shape[0], size = batch)
        Xk = X[idx,:]
        Yk = Y[idx,:]
        w = w_list[-1].reshape(-1,1) - lr / batch * (Xk.T @ (Xk @ w_list[-1].reshape(-1,1) - Yk))
        w_list.append(w.reshape(-1))
        
        loss_list.append(loss_batch(w,X,Y,batch))
        
    w_list = np.array(w_list)
    loss_list = np.array(loss_list)
    
    w_mean = np.mean(w_list[-500:], axis = 0)
    w_final = w_list[-1] - w_star
    
    return w_mean, w_final#, lr_list

def ideal_sol(X,Y):
    temp = np.dot(np.transpose(X), X)
    temp = np.linalg.inv(temp)
    temp = np.dot(temp, np.transpose(X))
    temp = np.dot(temp, Y)
    return temp

In [6]:
# cyclic lr

# base_lr = [1]
d = 100
K = 1000
T = 10000
mean_lr = [0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22]
delta = 0.05
stepsize = 10
batch = 15
alpha_list = []
w_star = ideal_sol(X,Y)

# pool = mp.Pool(mp.cpu_count())

for mean in mean_lr:
    random_vec = []
    
    for t in range(T):
        diff, _ = linearreg_cyc(d, X, Y, K, mean - delta, mean + delta, stepsize, batch, w_star)
#         diff = w - w_star
        random_vec.append(diff)
#         diff_list.append(diff)
    
#     random_vec = [pool.apply(linearreg_unif, args=(1,X,Y,K,base,max_lr,batch,w_star)) for t in range(T)]
    
    
    random_vec = np.array(random_vec)
    data = np.array(random_vec - np.mean(random_vec, axis = 0))
    alpha_list.append(alpha_estimator(100,data))
    
# pool.close()

### batch = 10

In [9]:
# delta = 0.05
alpha_list

[1.9991720893978229,
 2.011298303694093,
 2.0008479594466153,
 1.9874978527323601,
 1.5428267534633262,
 1.3720664714152018,
 1.269155771649106,
 1.1805871763417242,
 1.0950724369206228]

In [11]:
# delta = 0.04
alpha_list

[2.010257077015096,
 1.9959846996481365,
 1.9969430626601021,
 2.0066238209404803,
 1.6394994794382476,
 1.411773549362355,
 1.2847618578441176,
 1.194561979482453,
 1.1292451612829693]

In [13]:
# delta = 0.03
alpha_list

[2.016491455379517,
 2.0041872085830015,
 2.0042979040720605,
 2.008191658435959,
 1.745028696130977,
 1.4325435968283107,
 1.3165325322369972,
 1.2202257410979607,
 1.120917055548965]

In [15]:
# delta = 0.02
alpha_list

[2.000209075470449,
 2.007560240609068,
 2.012672405698283,
 1.9889764183431058,
 1.812992803698751,
 1.4773446996961004,
 1.3431437900133798,
 1.2307946095174598,
 1.135341374389148]

### batch = 5

In [6]:
# delta = 0.05
alpha_list

[0.959532605578557,
 0.9289879996308373,
 0.8703290561227857,
 0.8811398512176276,
 0.9908603951566133,
 0.9651772562390792,
 0.9565810347940601,
 0.9372771075289897,
 0.8873461148806024]

In [8]:
# delta = 0.05
# batcl = 2
alpha_list

[0.9773726729049528,
 0.9825678512204077,
 0.9700978916122925,
 0.9836473631496422,
 nan,
 nan,
 nan,
 nan,
 nan]

In [10]:
# batch = 7
# delta = 0.05
alpha_list

[1.3954468822681436,
 1.2920632144781692,
 1.161125644300706,
 1.0791494555255483,
 0.9878826752653216,
 0.9550900680831578,
 0.9284114637828675,
 0.8752488157381402,
 0.8986744000965463]

In [7]:
# batch = 15
# delta = 0.05
alpha_list

[2.003486981030688,
 1.9938059261214678,
 1.997572033406242,
 2.0001903070494245,
 2.006773515199711,
 2.0006186334523184,
 2.01266844327256,
 2.009809967978872,
 2.0119033679985248]